In [1]:
import socket

In [2]:
import threading
import http.server

In [ ]:
#head
import requests

res = requests.head('http://127.0.0.1:80/head.html')
print(res)

In [ ]:
#post
import requests
my_id = {'id': '107306052'}

#將資料加入 POST 請求中
r = requests.post('http://127.0.0.1:80/post.html', data = my_id)
print(r)

In [3]:
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

In [4]:
pars = ('127.0.0.1', 80) # you can change the server port to whatever you want
s.bind(pars)

In [5]:
s.listen(5)

In [6]:
def get_method(data):
    index = data.find("\r\n")
    if index == -1:
        return ""
    first_line = data[:index]
    arrs = first_line.split()
    method = arrs[0]
    return method

In [7]:
def get_path(data):
    index = data.find("\r\n")
    if index == -1:
        return ""
    first_line = data[:index]
    arrs = first_line.split()
    if len(arrs) != 3:
        return ""
    path = arrs[1]
    return path

In [8]:
def get_html_by_path(method,path):
    if method == 'GET':
        if path == '/get.html':
            return get_index()
        elif path=='/redirect.html':
            return get_redirect()
        elif path=='/notfound.html':
            return get_404()

html_string = """
    <html><body>good.html</body></html>
"""

notfound = """
    <html><body>404 Not Found</body></html>
"""

head_string = "HTTP/1.1 {status}\r\nServer: simple server\r\n"\
           "Content-Type: text/html; charset=utf-8\r\n"\
           "Content-Length: {length}\r\nConnection: close\r\n\r\n"

def get_html(status):
    length = len(html_string.encode())
    if status=='301 Moved Permanently':
        head = head_string1.format(status=status, length=length)
    else:
        head = head_string.format(status=status, length=length)
    return head

def get_redirect():
    return get_html('301 Moved Permanently')+html_string
    
def get_404():
    return ("HTTP/1.0 404 Not Found\r\nServer: simple server\r\n" \
           "Content-Type: text/html; charset=utf-8\r\n" \
           "Connection: close\r\n\r\n")

def get_index():
    return get_html("200 OK")+html_string

# a new thread is created for every new accepted client
# every new thread starts from the function below

In [9]:
def serveClient(clientsocket, address):
    
    # we need a loop to continuously receive messages from the client
    while True:
        # then receive at most 1024 bytes message and store these bytes in a variable named 'data'
        # you can set the buffer size to any value you like
        request = clientsocket.recv(1024).decode()
        
        path = get_path(request)

        msg = get_html_by_path(path)
        clientsocket.send(msg.encode())
        
        
        if request.startswith('GET /redirect.html'):
            clientsocket.send(b'HTTP/1.1 301 Moved Permanently\r\n')
            clientsocket.send(b'Location: http://127.0.0.1/get.html')
            clientsocket.close()
            break
        
        if request.startswith('HEAD /head.html'):
            clientsocket.send(b'HTTP/1.1 200 OK\r\n')
            clientsocket.close()
            break
                
        if request.startswith('POST /post.html'):
            clientsocket.send(b'HTTP/1.1 200 OK\r\n')
            clientsocket.send(b'<html><body>107306052</body></html>')
            clientsocket.close()
            break
        
        
        # we need some condition to terminate the socket
        # lets see if the client sends some termination message to the server
        # if so, then the server close the socket 
        clientsocket.close()

    s.close()

In [ ]:
while True:
    # accept a new client and get it's information
    (clientsocket, address) = s.accept()
    
    # create a new thread to serve this new client
    # after the thread is created, it will start to execute 'target' function with arguments 'args' 
    threading.Thread(target = serveClient, args = (clientsocket, address)).start()